In [14]:
!pip install fastapi uvicorn pydantic openai python-dotenv httpx nest_asyncio pyngrok


In [15]:
#libraries
import subprocess
import sys
import os
import time
import json
import requests
from multiprocessing import Process
import importlib

# List of packages required to run FastAPI and Ngrok
print("Installing the required packages(fastapi, uvicorn, pydantic, pyngrok, requests)...")
required_packages = "fastapi uvicorn pydantic pyngrok requests"
try:

    subprocess.check_call([sys.executable, "-m", "pip", "install", *required_packages.split()],
                           stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("The packages were installed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Package installation failed: {e}")
    sys.exit(1)

try:
    from pydantic import BaseModel, Field, conint, confloat
    from typing import List, Optional, Dict, Any, Type
    from fastapi import FastAPI
    from pyngrok import ngrok
    from pyngrok.exception import PyngrokError
    from uvicorn import Config, Server


    importlib.reload(sys.modules['pydantic'])
    importlib.reload(sys.modules['fastapi'])
    importlib.reload(sys.modules['uvicorn'])
    importlib.reload(sys.modules['pyngrok'])
except ImportError as e:

    print(f"Error importing libraries after installation: {e}")
    sys.exit(1)


Installing the required packages(fastapi, uvicorn, pydantic, pyngrok, requests)...
The packages were installed successfully.


In [16]:
# 2.(Self-Contained LLM Mock)
class ChatCompletion:

    def __init__(self, text: str):
        self.text = text

class LLMMockClient:

    def __init__(self, model: str, mock_response_text: str):
        self.model = model
        self.mock_response_text = mock_response_text

    async def generate_content_async(self, system_instruction: str, prompt: str) -> ChatCompletion:

        return ChatCompletion(self.mock_response_text)



# 3.Definition of data models (Pydantic Models)


class Client(BaseModel):
    name: str = Field(..., description="اسم العميل أو جهة الاتصال.")
    contact: str = Field(..., description="عنوان البريد الإلكتروني للعميل.")
    lang: str = Field(default="en", description="لغة العرض المطلوبة (en أو ar).")

class Item(BaseModel):
    sku: str = Field(..., description="رمز المنتج (Stock Keeping Unit).")
    qty: conint(gt=0) = Field(..., description="الكمية المطلوبة، يجب أن تكون أكبر من صفر.")
    unit_cost: confloat(gt=0) = Field(..., description="تكلفة الوحدة للمنتج (بالعملة المحددة).")
    margin_pct: conint(ge=0, le=100) = Field(..., description="نسبة الربح المئوية المراد تطبيقها على التكلفة.")

class QuotationRequest(BaseModel):
    client: Client
    currency: str = Field(..., description="رمز العملة (مثل SAR أو USD).")
    items: List[Item]
    delivery_terms: str = Field(..., description="شروط التسليم (مثل DAP).")
    notes: Optional[str] = Field(None, description="ملاحظات العميل الإضافية.")



# 4.FastAPI

app = FastAPI(title="خدمة توليد عروض الأسعار (Quotation Microservice)", version="1.0")
PORT = 8000
HOST = "127.0.0.1"


# 5.Quote generation endpoint (/quote)


async def generate_email_draft_mock(data: QuotationRequest, priced_items: List[Dict[str, Any]], grand_total: float) -> str:


    context = (
        f"Generate a professional quotation email draft in {data.client.lang} "
        f"for {data.client.name} ({data.client.contact}). "
        f"The total amount is {grand_total:.2f} {data.currency}. "
        f"Delivery terms: {data.delivery_terms}. "
        f"Key details: " +
        ", ".join([f"{item['sku']} x {item['qty']}" for item in priced_items])
    )

    # Arabic Language Fake Response Model
    if data.client.lang == "ar":
        mock_text = (
            "عزيزنا المهندس خالد العتيبي،\n\n"
            "تحية طيبة وبعد،\n\n"
            "يسرنا أن نقدم لكم عرض السعر التالي بناءً على طلبكم رقم (RFQ-1234).\n\n"
            f"الإجمالي الكلي لعرض السعر هو: {grand_total:,.2f} {data.currency}.\n"
            f"شروط التسليم: {data.delivery_terms}.\n"
            f"لقد تم التأكد من أن المواصفات تلبي متطلبات ترشيد استهلاك الطاقة حسب الملاحظة: '{data.notes}'.\n\n"
            "يرجى مراجعة الجدول المرفق للتفاصيل. لا تترددوا في التواصل لأي استفسار.\n\n"
            "مع خالص التحية،\n"
            "فريق المبيعات."
        )
    else:
        # Mock response template for English (to verify that the mock is working)
        mock_text = (
            f"Dear Eng. {data.client.name.split()[-1]},\n\n"
            f"Please find attached our quotation for your request (RFQ-1234).\n\n"
            f"The Grand Total for this quotation is: {grand_total:,.2f} {data.currency}.\n"
            f"Delivery terms: {data.delivery_terms}.\n"
            f"We noted your requirement: '{data.notes}'. We ensure compliance.\n\n"
            f"Best regards,\n"
            f"Sales Team."
        )

    # Using local LLMMockClient
    llm = LLMMockClient(
        model="gemini-2.5-flash-preview-09-2025",
        mock_response_text=mock_text
    )

    response: ChatCompletion = await llm.generate_content_async(
        system_instruction="You are a professional sales assistant for Alrouf Lighting.",
        prompt=context
    )

    return response.text

@app.post("/quote", response_model=Dict[str, Any])
async def create_quotation(request: QuotationRequest):

    priced_items = []
    grand_total = 0.0

    # 1. Calculating prices: Selling price = Cost / (1 - (Profit margin / 100))
    for item in request.items:

        selling_price = item.unit_cost / (1 - (item.margin_pct / 100.0))
        line_total = selling_price * item.qty

        priced_item = {
            "sku": item.sku,
            "qty": item.qty,
            "unit_cost": item.unit_cost,
            "margin_pct": item.margin_pct,
            "price_per_unit": selling_price,
            "line_total": line_total
        }
        priced_items.append(priced_item)
        grand_total += line_total

    # 2.Generating an email draft (using an LLM simulation)
    email_draft = await generate_email_draft_mock(request, priced_items, grand_total)

    return {
        "client_name": request.client.name,
        "currency": request.currency,
        "grand_total": grand_total,
        "priced_items": priced_items,
        "email_draft": email_draft
    }



# 6. Function to run Uvicorn in a separate process

def start_uvicorn_server():

    print(f"[Server Operation] Launching the FastAPI server on port{PORT}...")

    config = Config(app, host="0.0.0.0", port=PORT, log_level="info")
    server = Server(config)
    try:
        server.run()
    except Exception as e:
        print(f"[Server Operation] Critical error in Uvicorn: {e}")



# 7.Client Test Code - Direct Internal Connection


def test_client_code(host, port):
    """The POST request is sent to the quote service via direct local contact."""
    api_endpoint = f"http://{host}:{port}/quote"

    # Request for Quotation (RFQ) data (in Arabic)
    quotation_data = {
      "client": {
        "name": "المهندس خالد العتيبي",
        "contact": "khalid@client.com",
        "lang": "ar"
      },
      "currency": "SAR",
      "items": [
        {
          "sku": "ALR-SL-90W",
          "qty": 120,
          "unit_cost": 240.0,
          "margin_pct": 22
        },
        {
          "sku": "ALR-OBL-12V",
          "qty": 40,
          "unit_cost": 95.5,
          "margin_pct": 18
        }
      ],
      "delivery_terms": "DAP الدمام، 4 أسابيع",
      "notes": "العميل طلب مطابقة المواصفات مع متطلبات ترشيد لاستهلاك الطاقة."
    }


    print(f" Start executing the test request(POST {api_endpoint})...")

    headers = {
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(
            api_endpoint,
            data=json.dumps(quotation_data, ensure_ascii=False).encode('utf-8'),
            headers=headers,
            timeout=10
        )

        if response.status_code == 200:
            response.encoding = 'utf-8'
            result = response.json()

            print("\n Successfully responded! Status code: 200")



            print(f"Grand total: {result['grand_total']:,.2f} {result['currency']}")
            print(" Calculated items:")
            for item in result['priced_items']:
                print(f"  - SKU: {item['sku']} | Quantity: {item['qty']} | Price per unit: {item['price_per_unit']:.2f} {result['currency']} | الإجمالي: {item['line_total']:,.2f}")


            print("\n Email draft (generated byLLM Mock):")
            print("--------------------------------------------------")
            print(result['email_draft'])
            print("--------------------------------------------------")

        else:
            print(f"\n Failed to respond. Status code:{response.status_code}")
            print("--------------------------------------------------")
            print(f"Full text of the response:{response.text}")
            print("--------------------------------------------------")

    except requests.exceptions.ConnectionError:
        print("\n Connection error! Make sure the FastAPI server is running.localhost:8000.")
    except Exception as e:
        print(f"\n An unexpected error occurred during the test: {e}")



#8. The main entry point for stable operation

if __name__ == '__main__':
    server_process = Process(target=start_uvicorn_server)
    server_process.start()

    print("Wait 5 seconds to ensure the FastAPI server has started...")
    time.sleep(5)

    try:

        test_client_code(HOST, PORT)

    except Exception as e:
        print(f"\n Critical error during operation: {e}")
    finally:
        # 3. إيقاف عملية الخادم بشكل نظيف
        if server_process.is_alive():
            server_process.terminate()
            server_process.join()
            print("Server process has terminated Uvicorn.")

[Server Operation] Launching the FastAPI server on port8000...


INFO:     Started server process [28391]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


Wait 5 seconds to ensure the FastAPI server has started...


INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:41210 - "POST /quote HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [28391]


 Start executing the test request(POST http://127.0.0.1:8000/quote)...

 Successfully responded! Status code: 200
Grand total: 41,581.61 SAR
 Calculated items:
  - SKU: ALR-SL-90W | Quantity: 120 | Price per unit: 307.69 SAR | الإجمالي: 36,923.08
  - SKU: ALR-OBL-12V | Quantity: 40 | Price per unit: 116.46 SAR | الإجمالي: 4,658.54

 Email draft (generated byLLM Mock):
--------------------------------------------------
عزيزنا المهندس خالد العتيبي،

تحية طيبة وبعد،

يسرنا أن نقدم لكم عرض السعر التالي بناءً على طلبكم رقم (RFQ-1234).

الإجمالي الكلي لعرض السعر هو: 41,581.61 SAR.
شروط التسليم: DAP الدمام، 4 أسابيع.
لقد تم التأكد من أن المواصفات تلبي متطلبات ترشيد استهلاك الطاقة حسب الملاحظة: 'العميل طلب مطابقة المواصفات مع متطلبات ترشيد لاستهلاك الطاقة.'.

يرجى مراجعة الجدول المرفق للتفاصيل. لا تترددوا في التواصل لأي استفسار.

مع خالص التحية،
فريق المبيعات.
--------------------------------------------------
Server process has terminated Uvicorn.
